In [31]:
from policyengine_us import Simulation
import plotly.express as px
import pandas as pd
from policyengine_us import IndividualSim

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

# def make_exemption():
#     sim = IndividualSim(year=2021)
#     sim.add_person(name="head", age=25)
#     members = ["head"]
#     sim.add_tax_unit(name="tax_unit", members=members)
#     sim.add_spm_unit(name="spm_unit", members=members)
#     sim.add_household(name="household", members=members, state_code="NH")
#     sim.vary("charitable_cash_donations", max=800_000, step=1_000) 

def make(has_spouse, head_age, blind_head, disabled_head, spouse_age, blind_spouse, disabled_spouse, type):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=head_age, is_tax_unit_head=True, is_blind = blind_head, is_disabled = disabled_head)
    members = ["head"]
    if has_spouse:
        sim.add_person(name="spouse", age = spouse_age, is_tax_unit_head=False, is_blind = blind_spouse, is_disabled = disabled_spouse)
        members += ["spouse"]
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="NH")
    sim.vary("dividend_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            income=sim.calc("dividend_income")[0],
            nh_income_tax=sim.calc("nh_income_tax")[0].round(),
            mtr=sim.deriv(
                "nh_income_tax", "dividend_income", wrt_target="head"
            ),
            type = type
        )
    )

single = make(False, 30, False, False, 30, False, False, "single")
single_aged = make(False, 75, False, False, 60, False, False, "single and aged")
single_aged_blind = make(False, 75, True, False, 60, False, False, "single, aged and blind")
joint = make(True, 30, False, False, 30, False, False, "joint")
joint_aged_head_aged_spouse = make(True, 70, False, False, 70, False, False, "joint, both aged")

l = []
for i in [single, single_aged, single_aged_blind, joint, joint_aged_head_aged_spouse]:
    l.append(i)


In [32]:
df = pd.concat(l)
LABELS = dict(
    taxable_income="New Hampshire taxable income",
    nh_income_tax="New Hampshire income tax",
    mtr="Marginal tax rate",
)

GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

px.line(
    df,
    x="income",
    y="nh_income_tax",
    color="type",
    color_discrete_map={
        "single": GRAY,
        "single_and_aged": BLUE, 
    },
    template="plotly_white",
).update_layout(
    title="New Hampshire income tax, by houhold type",
    xaxis_title="New Hampshire income",
    yaxis_title="New Hampshire income tax",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
    width=800,
    height=600,
)

In [22]:
single = create_simulation(create_situation(False, False, False, False, False, False, False))
# taxable_income = single.calculate("nh_taxable_income")

AttributeError: module 'importlib' has no attribute 'metadata'